In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

# Step 1: Create a session and load the page
'''
url1 = 'https://www.nba.com/stats/players/pullup?PerMode=Totals'
url2 = 'https://www.nba.com/stats/players/catch-shoot?PerMode=Totals'
url3 = 'https://www.nba.com/stats/players/defense-dash-lt6?PerMode=Totals'
url4 = 'https://www.nba.com/stats/teams/isolation?PerMode=Totals'
url5 = 'https://www.nba.com/stats/players/transition?PerMode=Totals&dir=D&sort=POSS'
'''
cs ='https://www.nba.com/stats/teams/catch-shoot?PerMode=Totals'
pullup ='https://www.nba.com/stats/teams/pullup?PerMode=Totals'

handoff = 'https://www.nba.com/stats/teams/hand-off?PerMode=Totals'
iso ='https://www.nba.com/stats/teams/isolation?PerMode=Totals'
trans ='https://www.nba.com/stats/teams/transition?PerMode=Totals'
bh='https://www.nba.com/stats/teams/ball-handler?PerMode=Totals'
rollman = 'https://www.nba.com/stats/teams/roll-man?PerMode=Totals'
postup = 'https://www.nba.com/stats/teams/playtype-post-up?PerMode=Totals'
spotup = 'https://www.nba.com/stats/teams/spot-up?PerMode=Totals'
cut = 'https://www.nba.com/stats/teams/cut?PerMode=Totals'
offscreen = 'https://www.nba.com/stats/teams/off-screen?PerMode=Totals'
putbacks = 'https://www.nba.com/stats/teams/putbacks?PerMode=Totals'
misc = 'https://www.nba.com/stats/teams/playtype-misc?PerMode=Totals'

#url_list = [url1,url2,url3,url4,url5]
url_list=[handoff,iso,trans,bh,rollman,postup,spotup,cut,offscreen,putbacks,misc]
def get_tables(url_list):
    data = []
    for url in url_list:
        print(url)
        driver = webdriver.Chrome()
        driver.get(url)

        # Wait for the page to fully load
        driver.implicitly_wait(8)

        # Step 2: Parse HTML code and grab tables with Beautiful Soup
        soup = BeautifulSoup(driver.page_source, 'lxml')

        tables = soup.find_all('table')

        # Step 3: Read tables with Pandas read_html()
        dfs = pd.read_html(str(tables))

        #print(f'Total tables: {len(dfs)}')
        #print(dfs[2].head())
    
        driver.close()
        #return dfs
        df= dfs[-1]
        #drop = ['Unnamed: 16_level_1', 'Unnamed: 17_level_1', 'Unnamed: 18_level_1']
        #df.columns = df.columns.droplevel()
        #df = df.drop(columns = drop)
        data.append(df)
    return data

In [2]:
#url_list = [url1]
frames = get_tables(url_list)

https://www.nba.com/stats/teams/hand-off?PerMode=Totals
https://www.nba.com/stats/teams/isolation?PerMode=Totals
https://www.nba.com/stats/teams/transition?PerMode=Totals
https://www.nba.com/stats/teams/ball-handler?PerMode=Totals
https://www.nba.com/stats/teams/roll-man?PerMode=Totals
https://www.nba.com/stats/teams/playtype-post-up?PerMode=Totals
https://www.nba.com/stats/teams/spot-up?PerMode=Totals
https://www.nba.com/stats/teams/cut?PerMode=Totals
https://www.nba.com/stats/teams/off-screen?PerMode=Totals
https://www.nba.com/stats/teams/putbacks?PerMode=Totals
https://www.nba.com/stats/teams/playtype-misc?PerMode=Totals


In [3]:
frames[10]

,TEAM,GP,POSS,FREQ%,PPP,PTS,FGM,FGA,FG%,EFG%,FTFREQ%,TOVFREQ%,SFFREQ%,AND ONEFREQ%,SCOREFREQ%,PERCENTILE
0,Boston Celtics,17,110,5.9,0.74,82,11,19,57.9,73.7,30.9,51.8,2.7,0.0,38.2,100.0
1,Memphis Grizzlies,18,143,6.8,0.54,77,11,24,45.8,52.1,28.7,55.2,4.9,1.0,30.8,62.1
2,Detroit Pistons,19,130,6.0,0.58,76,12,33,36.4,39.4,25.4,50.0,6.2,1.0,33.1,82.8
3,Sacramento Kings,16,106,5.9,0.71,75,15,27,55.6,61.1,24.5,50.9,3.8,1.0,36.8,96.6
4,Cleveland Cavaliers,17,111,5.9,0.67,74,10,26,38.5,40.4,28.8,48.6,4.5,1.0,36.0,93.1
5,Brooklyn Nets,18,129,6.7,0.57,74,12,28,42.9,46.4,24.0,54.3,1.6,0.0,31.8,72.4
6,Miami Heat,18,134,6.8,0.55,74,13,29,44.8,46.6,22.4,56.7,3.0,1.0,30.6,65.5
7,Portland Trail Blazers,17,122,6.6,0.57,70,9,21,42.9,47.6,25.4,58.2,2.5,1.0,30.3,75.9
8,Utah Jazz,19,131,6.0,0.53,69,13,32,40.6,45.3,22.1,54.2,4.6,1.0,29.0,55.2
9,New York Knicks,18,105,5.1,0.65,68,12,31,38.7,43.5,25.7,46.7,3.8,2.0,35.2,89.7


In [4]:
#terms = ['data/teampullup.csv','data/teamcatchshoot.csv','data/teamundersix.csv','data/teamiso.csv','data/teamtransition.csv']
terms = ['playtype/handoff.csv','playtype/iso.csv','playtype/trans.csv','playtype/bh.csv','playtype/rollman.csv','playtype/postup.csv','playtype/spotup.csv',
         'playtype/cut.csv','playtype/offscreen.csv','playtype/putback.csv','playtype/misc.csv']
for i in range(len(terms)):
    df = frames[i]
    df.to_csv(terms[i],index = False)
   